In [ ]:
import pandas as pd

countries_df = pd.DataFrame({
    'country': ['Англия', 'Канада', 'США', 'Россия', 'Украина', 'Беларусь', 'Казахстан'],
    'population': [56.29, 38.05, 322.28, 146.24, 45.5, 9.5, 17.04],
    'square': [133396, 9984670, 9826630, 17125191, 603628, 207600, 2724902]
})

avg_pop = countries_df['population'] / countries_df['square'] * 1000000
print(avg_pop.mean())

In [ ]:
import pandas as pd

melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
#melb_data.head()
melb_df = melb_data.copy()
#melb_df.head()

melb_df.drop(['index','Coordinates'],axis=1,inplace=True)
#melb_df.head()

#total_rooms = melb_df['Rooms']+melb_df['Bedroom']+melb_df['Bathroom']
#display(total_rooms)

#melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms
#display(melb_df['MeanRoomsSquare'])

#diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
#sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
#melb_df['AreaRatio'] = diff_area/sum_area
#display(melb_df['AreaRatio'])


melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
#display(melb_df['Date'])

#month_sold = melb_df['Date'].dt.month
#print(month_sold)
#print('Min year sold:', month_sold.min())
#print('Max year sold:', month_sold.max())
#print('Mode year sold:', month_sold.mode()[0])
#melb_df['MonthSale'] = melb_df['Date'].dt.month
#melb_df['MonthSale'].value_counts(normalize=True)

delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
#display(delta_days)
#display(delta_days.dt.days)

#melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
#display(melb_df['AgeBuilding'])
#melb_df = melb_df.drop('YearBuilt', axis=1)
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df[(melb_df['WeekdaySale']==5)|(melb_df['WeekdaySale']==6)]
#display(weekend_count)
#print(melb_df['Address'].nunique())
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

street_types = melb_df['Address'].apply(get_street_type)
#display(street_types)
#display(street_types.value_counts())

popular_stypes =street_types.value_counts().nlargest(10).index
#print(popular_stypes)

melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
#display(melb_df['StreetType'])

melb_df = melb_df.drop('Address', axis=1)
#display(melb_df['WeekdaySale'])

def get_weekend(weekday):
    if weekday == 5 or weekday == 6:
        return 1
    else:
        return 0
    
melb_df['Weekend'] = melb_df['WeekdaySale'].apply(get_weekend)
#display(round(melb_df[melb_df['Weekend']==1]['Price'].mean(),2))

#print(melb_df['SellerG'])



popular_seller =melb_df['SellerG'].value_counts().nlargest(49).index
#print(popular_seller, 'lmao')

melb_df['SellerG'] = melb_df['SellerG'].apply(lambda x: x if x in popular_seller else 'other')
#print((melb_df[melb_df['SellerG']=='Nelson']['Price'].min())/(melb_df[melb_df['SellerG']=='other']['Price'].min()))

#melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
#display(melb_df['StreetType'])

# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(),melb_df[col].dtype) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
#display(unique_counts)

cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
#display(melb_df.info())

melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
display(melb_df['Type'])

In [ ]:
import pandas as pd
ufo_df = pd.read_csv('data/ufo.csv', sep=',')
ufo = ufo_df.copy()
ufo['Time'] = pd.to_datetime(ufo['Time'], dayfirst=False)
#display(ufo)
display(ufo['State'].drop_duplicates())
#display((ufo['Time'].dt.year).mode())
#display((((ufo[(['State']=='NE')(ufo['Time'].dt.date)].diff()).mean())))
ufo['Date'] = ufo['Time'].dt.date
print(ufo[ufo['State']=='NV']['Date'].diff().dt.days.mean())

In [ ]:
import pandas as pd

def get_experience(arg):
    """
    Напишите функцию get_experience(arg), аргументом которой является строка столбца с опытом работы. 
    Функция должна возвращать опыт работы в месяцах. Не забудьте привести результат к целому числу.
    """
    month_key_words = ['месяца', 'месяцев', 'месяц']
    year_key_words = ['год', 'года', 'лет']
    month = 0
    year = 0
    args_splitted = arg.split(' ')
    for i in range(len(args_splitted)):
        if args_splitted[i] in month_key_words:
            month = args_splitted[i-1]
        if args_splitted[i] in year_key_words:
            year = args_splitted[i-1]
    return int(year)*12+int(month)

if __name__ == '__main__':
    experience_col = pd.Series([
        'Опыт работы 8 лет 3 месяца',
        'Опыт работы 3 года 5 месяцев',
        'Опыт работы 1 год 9 месяцев',
        'Опыт работы 3 месяца',
        'Опыт работы 6 лет'
        ])
    experience_month = experience_col.apply(get_experience)
    print(experience_month)
    
